In [2]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file( 'test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

train_text = open(train_file,'rb').read().decode(encoding='utf-8')
test_text = open(test_file,'rb').read().decode(encoding='utf-8')



In [3]:
import pandas as pd

df_train = pd.DataFrame({
    'feature':[row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0],
    'target': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})


In [4]:
from konlpy.tag import Okt

#문장을 토큰화한것만 따로 저장
# tokenized_data =[]

# for i in df_train.iloc[:,0]:
#     tokenized_data.append(Okt().morphs( i ))

In [5]:
import pickle

#토근화된 데이터를 저장하였음
# with open('tokenized_data.pkl','wb') as f:
#     pickle.dump(tokenized_data,f)

#토큰화된 데이터 불러오기
# with open('tokenized_data.pkl','rb') as f:
#     tokenized_data_loaded = pickle.load(f)

# tokenized_data_loaded

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

df_train = df_train[::4]

vectorizer=TfidfVectorizer(tokenizer=Okt().morphs)
# X = vectorizer.fit_transform( df_train.iloc[:,0])
y = df_train.iloc[:,1]


In [7]:
# 불균형 데이터 확인
df_train[df_train.iloc[:,1]==0]  #18678 
df_train[df_train.iloc[:,1]==1]  #18822
df_train.shape

(37500, 2)

In [8]:
# 토큰화하고 빈도수(tfidf)에 기반한 벡터표현이 생성된 X를 따로저장
from scipy.sparse import save_npz ,load_npz

#vectorizer.fit_transform한것을 희소행렬로 저장
# save_npz('37500개_vectorized_data.npz', X)

#저장된 희소행렬을 불러오기
X_loaded = load_npz('37500개_vectorized_data.npz')

#이렇게 불러온 X_loaded는 바로 모델학습에 사용가능


In [9]:
import numpy as np
X_train_dense = np.expand_dims( X_loaded.toarray(), axis=-1 )
# X_train_dense.shape
# X_loaded.shape[1]
# X_loaded.toarray().shape[1]

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization , Input, Dropout, Embedding
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

In [ ]:
#flatten 사용하는것 알아보기

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.1

model = Sequential([
    encoder,
    Embedding(
        input_dim = len( encoder.get_vocabulary() ),
        output_dim = 64,
        mask_zero  = True),
    Bidirectional( LSTM(64) ),
    Dense( 64, activation = 'relu' ),
    Dense( 1, activation = 'softmax')
])

optimizer = Adam(lr = 0.001)
model.compile( optimizer = optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

#ROC커브 그래프확인해볼것

In [ ]:
ES = EarlyStopping(
    monitor='val_loss',
    mode = 'auto',
    restore_best_weights = True
)

MC = ModelCheckpoint(
    'best_RNN_model.h5',
    monitor='val_loss',
    save_best_only=True, mode='min'
)

LRS = LearningRateScheduler(scheduler)

model.fit(
    X_loaded.toarray(), y, 
    epochs= 10, 
    batch_size=500,
    validation_split=0.3, 
    callbacks=[ES,MC,LRS]
)
    

In [107]:
test = ['정말 별로 입니다. 이게뭐니']

# 학습된 TF-IDF 벡터라이저로 테스트 데이터 변환
X_test_sample = vectorizer.transform(test)
X_test_sample_dense = X_test_sample.toarray()

# 모델을 사용하여 예측 수행
predictions = model.predict(X_test_sample_dense)
print(predictions)

# 예측 결과 출력 (긍정: 1, 부정: 0)
print("예측 결과:", np.round( predictions ).astype(int))

1/1 [==============================] - 0s 49ms/step
[[0.01700829]]
예측 결과: [[0]]


In [25]:
# model.save('기본 DNN 적용모델.h5')